In [ ]:
import peewee
# import peewee
from SECRETS import sql_account, sql_pw


# pg_db = peewee.PostgresqlDatabase('my_app', user=sql_account, password=sql_pw,
#                                   host='localhost', port=5432)



from peewee import *

# db = SqliteDatabase('people.db')
pg_db = peewee.PostgresqlDatabase('spacetraders',
                                  user=sql_account,
                                  password=sql_pw,
                                  host='localhost',
                                  port=5432)


In [1]:
import peewee
# import peewee
from SECRETS import sql_account, sql_pw


# pg_db = peewee.PostgresqlDatabase('my_app', user=sql_account, password=sql_pw,
#                                   host='localhost', port=5432)



from peewee import *

# db = SqliteDatabase('people.db')
pg_db = peewee.PostgresqlDatabase('spacetraders',
                                  user=sql_account,
                                  password=sql_pw,
                                  host='localhost',
                                  port=5432)

class Person(Model):
    name = CharField()
    birthday = DateField()

    class Meta:
        database = pg_db # This model uses the "people.db" database.



In [20]:
pg_db = peewee.PostgresqlDatabase('spacetraders',
                                  user=sql_account,
                                  password=sql_pw,
                                  host='localhost',
                                  port=5432)


class Pet(Model):
    owner = ForeignKeyField(Person, backref='pets')
    name = CharField()
    animal_type = CharField()

    class Meta:
        database = pg_db # this model uses the "people.db" database

In [13]:
pg_db.close()

True

In [21]:
#jupyter probably weirds this a bit, but reuse if open fixes it
pg_db.connect(reuse_if_open=True)

pg_db.create_tables([Person, Pet])

pg_db.close()

True

In [6]:
from datetime import date
uncle_bob = Person(name='Bob', birthday=date(1960, 1, 15))
uncle_bob.save() # bob is now stored in the database
# Returns: 1


grandma = Person.create(name='Grandma', birthday=date(1935, 3, 1))
herb = Person.create(name='Herb', birthday=date(1950, 5, 5))

In [7]:
grandma.name = 'Grandma L.'
grandma.save()  # Update grandma's name in the database.
# Returns: 1

1

In [8]:
bob_kitty = Pet.create(owner=uncle_bob, name='Kitty', animal_type='cat')
herb_fido = Pet.create(owner=herb, name='Fido', animal_type='dog')
herb_mittens = Pet.create(owner=herb, name='Mittens', animal_type='cat')
herb_mittens_jr = Pet.create(owner=herb, name='Mittens Jr', animal_type='cat')

In [9]:
herb_mittens.delete_instance() # he had a great life
# Returns: 1

1

In [11]:
grandma = Person.select().where(Person.name == 'Grandma L.').get()
print(grandma)

2


In [13]:
grandma = Person.get(Person.name == 'Grandma L.')
print(grandma)
#this is only returning the id?

#ah okay, it's like attributes
print(grandma.name)


2
Grandma L.


In [14]:

for person in Person.select():
    print(person.name)


Bob
Herb
Grandma L.


In [15]:
query = Pet.select().where(Pet.animal_type == 'cat')
for pet in query:
    print(pet.name, pet.owner.name)

# There is a big problem with the previous query: because we are accessing pet.owner.name and we did not select this relation in our original query, peewee will have to perform an additional query to retrieve the pet’s owner. This behavior is referred to as N+1 and it should generally be avoided.
#
# For an in-depth guide to working with relationships and joins, refer to the Relationships and Joins documentation.


Kitty Bob
Mittens Jr Herb


In [18]:
for person in Person.select().order_by(Person.birthday.desc()):
    print(person.name, person.birthday)
    print(type(person.name), type(person.birthday))

Bob 1960-01-15
<class 'str'> <class 'datetime.date'>
Herb 1950-05-05
<class 'str'> <class 'datetime.date'>
Grandma L. 1935-03-01
<class 'str'> <class 'datetime.date'>


In [19]:
for person in Person.select():
    print(person.name, person.pets.count(), 'pets')

Bob 1 pets
Herb 2 pets
Grandma L. 0 pets
